# Get data and packages

In [1]:
%%capture
%%bash
pip install jupyter_contrib_nbextensions
pip install jupyter_nbextensions_configurator
jupyter contrib nbextension install --user
jupyter nbextensions_configurator enable --user

jupyter nbextension enable codefolding/main
jupyter nbextension enable scratchpad/main
jupyter nbextension enable execute_time/ExecuteTime
jupyter nbextension enable autosavetime/main
pip install -U pip dask numpy fsspec>=0.3.3 tqdm pyarrow


In [6]:
%%capture
!pip install -U pip datatable vaex-core vaex-hdf5 vaex-arrow

In [ ]:
%%capture
!aws s3 cp s3://xdss-public-datasets/demos/taxi_1B.hdf5 datasets/taxi_1B.hdf5

In [1]:
import gc
import numpy as np
import pandas as pd
import warnings
import time
import gc
import os

instance_type = 'c5d2xlarge' # change this
results_bucket = f"s3://vaex-sagemaker-demo/benchmarks" # change this

name = 'datatable'
data_path = 'datasets/taxi_1B.hdf5'
output_file = f'{name}_{instance_type}_results.csv'
results_path = f"results/{output_file}"
results_bucket = f"{results_bucket}/{output_file}" 
benchmarks = {}
single_repetition = 1
statistic_repetition = 5

long_min = -74.05
long_max = -73.75
lat_min = 40.58

long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90


def get_results(benchmarks, name):
    results = pd.DataFrame.from_dict(benchmarks, orient='index')
    results.columns = [name]
    return results

def persist():
    gc.collect()
    get_results(benchmarks, name).to_csv(results_path)
    os.system(f"aws s3 cp {results_path} {results_bucket}")
    
def benchmark(f, df, name, repetitions=1, **kwargs):
    times = []
    for i in range(repetitions):
        start_time = time.time()
        ret = f(df, **kwargs)
        times.append(time.time()-start_time)
    benchmarks[name] = np.mean(times)
    persist()
    print(f"{name} took: {benchmarks[name]}")
    return benchmarks[name]


!mkdir -p results
!mkdir -p datasets
print(f"test for {single_repetition} repetitions for join and groupby and {statistic_repetition} repetitions for statistics")

test for 1 repetitions for join and groupby and 5 repetitions for statistics


# Benchmark

In [2]:
import vaex
import numpy as np
import datatable as dt
from datatable import f, math

In [3]:
def read_file(data=None):
    vdf = vaex.open(data_path)
    columns = {}
    for name in vdf.get_column_names():
        data = vdf.columns[name]
        if data.dtype == str:
            pass  # skip strings
        elif data.dtype.kind == 'f':
            # datatable is picky about <f4 format
            columns[name] = data.view(np.float32)
        elif data.dtype.kind == 'i':
            columns[name] = data
        else:
            pass  # ignore non int and float
    return dt.Frame(**columns)

# Load data
data = read_file(data=None)
print(f"size: {data.shape[0]} with {data.shape[1]} columns")

size: 1173057927 with 14 columns


In [4]:
benchmark(read_file, df=data, name='read_file', repetitions=statistic_repetition)

read_file took: 0.0057408809661865234


0.0057408809661865234

In [5]:
def count(df=None):
    return df.shape[0]

benchmark(count, df=data, name='count', repetitions=statistic_repetition)

count took: 1.7642974853515626e-06


1.7642974853515626e-06

In [6]:
def mean(df):
    return df[:, dt.mean(dt.f.fare_amount)]

benchmark(mean, df=data, name='mean', repetitions=statistic_repetition)

mean took: 8.807757329940795


8.807757329940795

In [7]:
def standard_deviation(df):
    return df[:, dt.sd(dt.f.fare_amount)]

benchmark(standard_deviation, df=data, name='standard deviation', repetitions=statistic_repetition)

standard deviation took: 7.904828929901123


7.904828929901123

To calculate the time when using two columns, we can't return the response since it will get into memroy and break, so we run a mean calculation on it, and then remove the time it took to run the mean.

In [8]:
def mean_of_sum(df):
    return df[:, dt.mean(f.fare_amount + f.trip_distance)]

benchmark(mean_of_sum, df=data, name='sum columns mean', repetitions=statistic_repetition)

sum columns mean took: 17.174331140518188


17.174331140518188

In [9]:
def sum_columns(df):
    return df[:, f.fare_amount + f.trip_distance]

benchmark(sum_columns, df=data, name='sum columns', repetitions=statistic_repetition)

sum columns took: 2.9754638671875e-05


2.9754638671875e-05

In [10]:
def mean_of_product(df):
    return df[:, dt.mean(f.fare_amount * f.trip_distance)]

benchmark(mean_of_product, df=data, name='product columns mean', repetitions=statistic_repetition)

product columns mean took: 14.600020599365234


14.600020599365234

In [11]:
def product(df):
    return df[:, f.fare_amount * f.trip_distance]

benchmark(product, df=data, name='product columns', repetitions=statistic_repetition)

product columns took: 2.93731689453125e-05


2.93731689453125e-05

In [12]:
from datatable import math
# Memorry crash
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = f.pickup_longitude
    phi_1 = f.pickup_latitude
    theta_2 = f.dropoff_longitude
    phi_2 = f.dropoff_latitude
    temp = (math.sin((theta_2-theta_1)/2*math.pi/180)**2
           + math.cos(theta_1*math.pi/180)*math.cos(theta_2*math.pi/180) * math.sin((phi_2-phi_1)/2*math.pi/180)**2)
    expr = 2 * math.atan2(math.sqrt(temp), math.sqrt(1-temp))
    return df[:, dt.mean(expr)]

# benchmark(mean_of_complicated_arithmetic_operation, df=data, name='arithmetic operation mean', repetitions=statistic_repetition)
benchmarks['arithmetic operation'] =  np.nan
benchmarks['arithmetic operation mean'] =  np.nan

In [ ]:
# Memorry crash
from datatable import f, count

def value_counts(df):
    return df[:,count(f.passenger_count),'passenger_count']

# benchmark(value_counts, df=data, name='value counts', repetitions=statistic_repetition)
benchmarks['value counts'] =  np.nan

In [ ]:
def groupby_statistics(df):
    aggs = {
            'fare_amount_mean': dt.mean(f.fare_amount),
            'fare_amount_std': dt.sd(f.fare_amount),
            'tip_amount_mean': dt.mean(f.tip_amount),
            'tip_amount_std': dt.sd(f.tip_amount),
        }
    return df[:, aggs, dt.by(f.passenger_count)]

benchmark(groupby_statistics, df=data, name='groupby statistics', repetitions=single_repetition)

In [ ]:
other = groupby_statistics(data)

In [ ]:
def join(df, other):
    # like vaex and dask, no precomputed index
    other.key = 'passenger_count'
    return df[:,:,dt.join(other)]

benchmark(join, data, name='join', repetitions=single_repetition, other=other)

In [ ]:
def join_count(df, other):
    # like vaex and dask, no precomputed index
    other.key = 'passenger_count'
    return df[:,:,dt.join(other)].shape[0]

benchmark(join_count, data, name='join count', repetitions=single_repetition, other=other)

## Filtered data

In [ ]:
For the entire dataaset, non of this runs

Dask is not build to run on filter data like you would normally, so we will apply the same strategy

In [ ]:
print(f"Prepare filtered data and deleted {gc.collect()} MB")

In [ ]:
def filter_data(df):
    expr_filter = (f.pickup_longitude > long_min)  & (f.pickup_longitude < long_max) & \
              (f.pickup_latitude > lat_min)    & (f.pickup_latitude < lat_max) & \
              (f.dropoff_longitude > long_min) & (f.dropoff_longitude < long_max) & \
              (f.dropoff_latitude > lat_min)   & (f.dropoff_latitude < lat_max)
    return df[expr_filter,:]

benchmark(filter_data, data, name='filter data', repetitions=single_repetition)

In [ ]:
filterd = filter_data(data)

del data
print(f"cleaned {gc.collect()} mb")

benchmark(filter_data, filterd, name='filterd count', repetitions=statistic_repetition)
benchmark(mean, filterd, name='filterd mean', repetitions=statistic_repetition)
benchmark(standard_deviation, filterd, name='filtered standard deviation', repetitions=statistic_repetition)
benchmark(mean_of_sum, filterd, name ='filtered sum columns mean', repetitions=statistic_repetition)
benchmark(sum_columns, df=filterd, name='filtered sum columns', repetitions=statistic_repetition)
benchmark(mean_of_product, filterd, name ='filterd product columns mean', repetitions=statistic_repetition)
benchmark(product, df=filterd, name='filterd product columns', repetitions=statistic_repetition)
benchmark(mean_of_complicated_arithmetic_operation, filterd, name='filterd arithmetic operation mean', repetitions=statistic_repetition)
benchmarks['filterd arithmetic operation'] =  np.nan
benchmark(value_counts, filterd, name ='filtered value counts', repetitions=statistic_repetition)
benchmark(groupby_statistics, filterd, name='filtered groupby statistics', repetitions=single_repetition)
other = groupby_statistics(filterd)
benchmark(join, filterd, name ='filtered join', repetitions=single_repetition, other=other)
benchmark(join_count, filterd, name='filtered join count', repetitions=single_repetition, other=other)
print('datatable!')
print(get_results(benchmarks,'datatable'))